In [22]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [23]:
#Constants
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type":"application/json"}
MODEL = "llama3.2"

In [24]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class Website:
    def __init__(self, url):
        self.url = url
        req = requests.get(self.url, headers=headers)
        soup = BeautifulSoup(req.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for irr in soup.body(["script","img","input","button","style"]):
            irr.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [25]:
system_prompt = "You are a website analyzer assistant \
Provide a short summary of he website, ignore navigation related texts \
Respond in Markdown"

In [26]:
def user_prompt_for(website):
    user_prompt = f"You are looking at {website.title}"
    user_prompt += " Provide a short summary about the contents of this website, If there are news or articles or blogs present\
    in the website summarize that too, the contents of the website are as follows "
    user_prompt += f"\n\n ${website.text}"
    return user_prompt

In [27]:
def messages(website):
    return [
        {"role":"system","content":system_prompt},
        {"role":"user","content":user_prompt_for(website)}
    ]

In [ ]:

def summarizer(url):
    website = Website(url)
    res = requests.post(OLLAMA_API, json={"model":MODEL,"messages":messages(website),"stream":False}, headers=HEADERS)
    display(Markdown(res.json()["message"]["content"]))
summarizer("https://www.masaischool.com/")